In [ ]:
import pandas as pd

# Load the clean dataset with explicit encoding
df = pd.read_csv('clean_kaggle_data_2022.csv', encoding='ISO-8859-1')
df_head = pd.read_csv('clean_kaggle_data_2022.csv', encoding='ISO-8859-1', nrows=1)

# Get the first row as column names
column_names=df_head.columns.tolist()

# Drop the first two rows (header and unit information)
df = df[2:]

# Set the column names
df.columns = column_names

# Iterate through each column starting from the second column (index 1)
response_tally = []

for column in df.columns[1:]:
    responses = df[column].value_counts().reset_index()
    responses.columns = ['Response', 'Count']
    responses['Column'] = column
    response_tally.append(responses)

# Concatenate the response tally dataframes
result_df = pd.concat(response_tally, ignore_index=True)

# Save the result to an Excel file
result_df.to_excel('numberOfResponsesPerClass.xlsx', index=False)


In [66]:
import pandas as pd

# Load the Excel file into a DataFrame
df = pd.read_excel('response_tally.xlsx')

# Initialize an empty dictionary to hold string tallies
column_tally = {}

# Loop through each row in the "Column" column
for item in df['Column']:
    if item in column_tally:
        column_tally[item] += 1                        
    else:
        column_tally[item] = 1

# Convert the column tally dictionary to a DataFrame
output_df = pd.DataFrame(list(column_tally.items()), columns=['Column', 'Tally'])

# Save the DataFrame to a new Excel file
output_df.to_excel('numberOfclassesPerQuestion.xlsx', index=False)


In [5]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from category_encoders import BinaryEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import numpy as np

# Load the dataset
df = pd.read_csv('clean_kaggle_data_2022.csv')

# Handling missing values by filling 'null' with 'unknown'
df.fillna('unknown', inplace=True)

# Encoding categorical features

# Initialize label encoder
label_encoder = LabelEncoder()

# List of questions to be label encoded
label_encoding_questions = ['Q2', 'Q11', 'Q16', 'Q25', 'Q26', 'Q30', 'Q43']
for question in label_encoding_questions:
    # Encode only if the column exists in the DataFrame
    if question in df.columns:
        df[question] = label_encoder.fit_transform(df[question])

# Binary encoding for Q9
binary_encoder = BinaryEncoder(cols=['Q9'])
df = binary_encoder.fit_transform(df)

# Remove Q5 as per the instructions
df.drop('Q5', axis=1, inplace=True)

# One-hot encoding for the rest of the columns, except already encoded ones
one_hot_encoding_questions = [col for col in df.columns if col not in label_encoding_questions and col != 'Q9']
df = pd.get_dummies(df, columns=one_hot_encoding_questions)

# Handle the target 'Q29_buckets'
target_columns = [col for col in df.columns if 'Q29_buckets_' in col]
if target_columns:
    # Revert one-hot encoding for the target
    y = df[target_columns].idxmax(axis=1).str.replace('Q29_buckets_', '')
    X = df.drop(columns=target_columns)
else:
    X = df.drop('Q29_buckets', axis=1, errors='ignore')
    y = df['Q29_buckets']

# Splitting the dataset into the Training set and Test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Using RandomForest to measure feature importance
def encode_salary(salary):
    """
    Convert salary range strings into their midpoint values for regression modeling.
    """
    if '-' in salary:
        lower, upper = salary.split('-')
        lower = int(lower.replace(',', ''))
        upper = int(upper.replace(',', ''))
        return (lower + upper) / 2
    elif salary == '>300,000':
        return 350000  # or you can use 300000
    elif salary == 'unknown':
        return np.nan  # you can later decide to fill these NaNs or drop them
    else:
        return np.nan

# Apply the function to the target variable
y_train = y_train.apply(encode_salary)
y_test = y_test.apply(encode_salary)

nan_indices = y_train[y_train.isna()].index
y_train.drop(nan_indices, inplace=True)
X_train.drop(nan_indices, inplace=True)

# Now, fit the RandomForest
rf = RandomForestRegressor(n_estimators=100)
rf.fit(X_train, y_train)

# Get the importance of each feature
feature_importances = rf.feature_importances_

# Create a DataFrame for the importances
features_df = pd.DataFrame({
    'Features': X_train.columns,
    'Importance': feature_importances
})

# Sort the DataFrame based on importance
features_df = features_df.sort_values(by='Importance', ascending=False)

# Plotting the feature importances
plt.figure(figsize=(15,5))
sns.barplot(data=features_df.iloc[:15], x='Features', y='Importance') # showing top 15 features
plt.xticks(rotation=90)
plt.title("Top 15 Important Features")
plt.show()


C:\Users\souren\AppData\Local\Temp\ipykernel_22844\2203218641.py:11: DtypeWarning: Columns (0,15,43,57,73,88,104,118,126,132,170,200,215,248,272,281,294) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('clean_kaggle_data_2022.csv')
C:\Users\souren\AppData\Local\Temp\ipykernel_22844\2203218641.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'unknown' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('unknown', inplace=True)


NameError: name 'np' is not defined

In [19]:
from sklearn.preprocessing import StandardScaler

# Initialize a scaler
scaler = StandardScaler()

# Fit the scaler on the training data and transform
X_train_scaled = scaler.fit_transform(X_train)

# Transform the test data
X_test_scaled = scaler.transform(X_test)


In [20]:
import mord

# Initialize an ordinal logistic regression model
ord_logit = mord.LogisticAT() # LogisticAT is one of the types of ordinal logistic regression

In [26]:
from sklearn.model_selection import cross_val_score

# Updated mapping of salary midpoints to integer labels
salary_to_int = {
    4999.5: 0,
    14999.5: 1,
    24999.5: 2,
    34999.5: 3,
    44999.5: 4,
    54999.5: 5,
    64999.5: 6,
    74999.5: 7,
    84999.5: 8,
    94999.5: 9,
    112499.5: 10,
    137499.5: 11,
    174999.5: 12,
    249999.5: 13,
    350000.0: 14,  # This represents >300,000 as it's the upper limit
    'unknown': 15
}

# Map the encoded salaries to integers
y_train_int = y_train.map(salary_to_int)
y_test_int = y_test.map(salary_to_int)

# Ensure no NaN values in the mapped labels
assert y_train_int.isna().sum() == 0, "NaN values exist in y_train_int"
assert y_test_int.isna().sum() == 0, "NaN values exist in y_test_int"

# Apply ordinal logistic regression on the integer labels
accuracies = cross_val_score(ord_logit, X_train_scaled, y_train_int, cv=10)

# Average and variance of accuracy
avg_accuracy = accuracies.mean()
var_accuracy = accuracies.var()


In [2]:
from sklearn.metrics import make_scorer, f1_score
from sklearn.model_selection import GridSearchCV
import joblib  # Import joblib for model saving

# Hyperparameter grid
param_grid = {
    'alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
    'max_iter': [100, 500, 1000, 5000]
}

# Initialize GridSearchCV with f1_score as the scoring metric
grid_search = GridSearchCV(
    ord_logit,
    param_grid,
    cv=10,
    scoring=make_scorer(f1_score, average='micro'),  # f1_score micro is a good general choice
    verbose=1
)

# Fit the grid search to the data
grid_search.fit(X_train_scaled, y_train_int)

# Extract best hyperparameters and the best model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# Print out the best parameters
print("Best parameters found: ", best_params)

# Save the best model to disk using joblib
joblib.dump(best_model, 'best_ordinal_logit_model.joblib')

# Optional: Load the model later using joblib
# loaded_model = joblib.load('best_ordinal_logit_model.joblib')


KeyboardInterrupt: 

In [1]:
# Assuming you've performed grid search and got the best_model

# Extract coefficients (since it's scalar, it will be the same for all features)
coef_scalar = best_model.coef_

# Feature names
features = X_train.columns

# Generate a list with the scalar repeated for all features (since it's constant for all)
importance = [coef_scalar] * len(features)

# Plot
plt.figure(figsize=(12,8))
plt.title('Feature Importance (using coefficient magnitude)')
plt.barh(features, importance, align='center')
plt.xlabel('Coefficient Magnitude')
plt.show()


NameError: name 'best_model' is not defined

In [25]:
# Identify the unmatched categories in y_train
unmatched_categories = y_train[y_train_int.isna()].unique()

print("Unmapped categories:", unmatched_categories)



Unmapped categories: [ 34999.5 112499.5  94999.5   4999.5 249999.5  44999.5  24999.5 174999.5
  74999.5  54999.5  14999.5  64999.5 350000.  137499.5  84999.5]
